In [ ]:
# Imports
import openai
import requests
from bs4 import BeautifulSoup
import json
import datetime
import my_target_job
import my_secrets

In [ ]:
# Set OpenAI API key
openai.api_key = my_secrets.OPENAI_API_KEY

In [ ]:
# Get job description from URL

# Send a GET request to the URL
url_response = requests.get(my_target_job.job_description_url)

# Extract the HTML content from the response
html_content = url_response.text

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, "html.parser")

# Extract the text from the webpage
job_description = soup.get_text()

# Remove whitespace characters
job_description = " ".join(job_description.split())  

print(job_description)

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,  # degree of randomness of GPT response
    )
    return response.choices[0].message["content"]

In [ ]:
# Extract information from job description

# Delimiter used to mark the start and end of the job description
delimiter = "####"

# Prompt for extracting information from job description
extraction_prompt = f"""
Deine Aufgabe ist es, Informationen über Arbeitgeber, Stellenbezeichnung, Anforderungen, Aufgaben,
Kontaktperson und Adresse aus der Stellenbeschreibung, die mit {delimiter} Zeichen markiert ist, 
zu extrahieren.

Formattiere deine Antwort als Python Dictionary mit "Arbeitgeber", "Stellenbezeichnung", 
"Anforderungen", "Aufgaben", "Kontaktperson", und "Adresse" als Schlüssel. 

Formattiere die "Anforderungen" und "Aufgaben" jeweils als Liste. 

Wenn die Information in der Stellenbeschreibung fehlt, verwende "unbekannt" als Wert. 

Antworte so kurz wie möglich. 

Stellenbeschreibung: {delimiter}{job_description}{delimiter}
"""

# Get extraction response
extraction_response_str = get_completion(extraction_prompt)

# Convert extraction response from str to json
extraction_response = json.loads(extraction_response_str)

# Show extracted information
print(extraction_response)

In [ ]:
# Create cover letter

# Cover letter prompt
cover_letter_prompt = f"""
Deine Aufgabe ist es, ein professionelles Bewerbungsanschreiben zu erstellen.

Adressiere das Anschreiben an folgenden Arbeitgeber, Adresse, Stelle und Kontaktperson:
Arbeitgeber: {extraction_response["Arbeitgeber"]}
Adresse: {extraction_response["Adresse"]}
Stelle: {extraction_response["Stellenbezeichnung"]}
Kontaktperson: {extraction_response["Kontaktperson"]}

Verwende folgenden Absender:
Name: {my_secrets.name}
Adresse: {my_secrets.adress}
Telefonnummer: {my_secrets.phone}
E-Mail: {my_secrets.email}

Verwende im Briefkopf des Anschreibens Ort und Datum. 
Verwende den Ort aus: {my_secrets.adress}
Verwende das folgende, aktuelle Datum im deutschen Datumsformat: {datetime.date.today()}

Beschreibe inwiefern die Anforderungen und Aufgaben erfüllt werden durch die 
Ausbildung, Arbeitserfahrung, Kompetenzen und Motivation. 
Verwende hierfür die folgenden Informationen:
Anforderungen: {extraction_response["Anforderungen"]}
Aufgaben: {extraction_response["Aufgaben"]}
Ausbildung: {my_secrets.education}
Arbeitserfahrung: {my_secrets.work_experience}
Kompetenzen: {my_secrets.skills}
Motivation: {my_target_job.motivation}

Nenne folgende Gehaltsvorstellung und mögliches Eintrittsdatum:
Gehaltsvorstellung : {my_secrets.salary_expectations}
Mögliches Eintrittsdatum: {my_secrets.possible_start_date}

Schreibe in einem professionellen, präzisen und kompakten Ton.

Unterschreibe das Anschreiben als {my_secrets.name}.
"""

# Get 2 cover letter suggestions
for i in range(2):
    print(f"Anschreiben {i+1}")
    print("=" * 5)
    cover_letter_response = get_completion(cover_letter_prompt, temperature=0.7)
    print(cover_letter_response)
    print("=" * 30)